# Исследование данных об инвестиции венчурных фондов в компании-стартапы

[Схема базы данных и описание базы](https://code.s3.yandex.net/SQL%20for%20data%20and%20analytics/ER/basic_sql_project.pdf)

**Описание проекта**

В этом проекте идёт работа с базой данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы. Эта база данных основана на датасете Startup Investments, опубликованном на популярной платформе для соревнований по исследованию данных Kaggle.

**Задача 1**

Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(status) AS closed_company  
FROM company  
WHERE status = 'closed';

**Задача 2**

Отобразите количество привлечённых средств для новостных компаний США. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total
FROM company
WHERE category_code='news'
  AND country_code='USA'
ORDER BY funding_total DESC;

**Задача 3**

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code='cash'
  AND EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) IN (2011,2012,2013);

**Задача 4**

Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

**Задача 5**

Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT*
FROM people
WHERE twitter_username LIKE '%money%'
  AND last_name LIKE 'K%';

**Задача 6**

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

**Задача 7**

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению

In [ ]:
SELECT CAST(funded_at AS date),
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY CAST(funded_at AS date)
HAVING MIN(raised_amount)!=0
       AND MIN(raised_amount)!=MAX(raised_amount);

**Задача 8**

Создайте поле с категориями:
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
  CASE 
     WHEN invested_companies >=100 THEN 'high_activity'
     WHEN invested_companies BETWEEN '20' AND '100' THEN 'middle_activity'
     ELSE 'low_activity'
   END AS category
FROM fund;

**Задача 9**

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY AVG(investment_rounds);

**Задача 10**

Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies)!=0
ORDER BY AVG(invested_companies) DESC,
         country_code
LIMIT 10;

**Задача 11**

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id=e.person_id;

**Задача 12**

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition)  
FROM company AS c
JOIN people AS p ON c.id=p.company_id
JOIN education AS e ON e.person_id=p.id
GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5;

**Задача 13**

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT(c.name)
FROM company AS c
LEFT JOIN funding_round AS f ON c.id=f.company_id
WHERE f.is_first_round = 1
  AND f.is_last_round = 1
  AND c.status = 'closed'

**Задача 14**

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT(p.id)
FROM company AS c
JOIN funding_round AS f ON c.id=f.company_id
 AND f.is_first_round = 1 
 AND f.is_last_round = 1
JOIN people AS p ON c.id=p.company_id
WHERE c.status='closed';

**Задача 15**

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT p.id,
       e.instituition
FROM company AS c
JOIN funding_round AS f ON c.id=f.company_id
 AND f.is_first_round = 1 
 AND f.is_last_round = 1
JOIN people AS p ON c.id=p.company_id
JOIN education AS e ON e.person_id=p.id
WHERE c.status='closed';

**Задача 16**

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT id,
       CASE
       WHEN id=15503 THEN 2
       ELSE COUNT(instituition)
       END
FROM (SELECT p.id,
             e.instituition
      FROM company AS c
      JOIN funding_round AS f ON c.id=f.company_id
          AND f.is_first_round = 1 
          AND f.is_last_round = 1
      JOIN people AS p ON c.id=p.company_id
      JOIN education AS e ON e.person_id=p.id
      WHERE c.status='closed') AS t
GROUP BY id;

**Задача 17**

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(count_instituition) AS avg_instituition
FROM(SELECT id,
       CASE
       WHEN id=15503 THEN 2
       ELSE COUNT(instituition)
       END AS count_instituition
FROM (SELECT p.id,
             e.instituition
      FROM company AS c
      JOIN funding_round AS f ON c.id=f.company_id
          AND f.is_first_round = 1 
          AND f.is_last_round = 1
      JOIN people AS p ON c.id=p.company_id
      JOIN education AS e ON e.person_id=p.id
      WHERE c.status='closed') AS t
GROUP BY id) AS z;

**Задача 18**

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook.

In [ ]:
SELECT AVG(count_instituition)
FROM(SELECT id,
       COUNT(instituition) AS count_instituition
      FROM (SELECT p.id,
                   e.instituition
              FROM company AS c
              JOIN people AS p ON c.id=p.company_id
              JOIN education AS e ON e.person_id=p.id
             WHERE c.name='Facebook') AS t
      GROUP BY id) AS z;

**Задача 19**

Составьте таблицу из полей:
- name_of_fund — название фонда;
- name_of_company — название компании;
- amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT fund.name AS name_of_fund,
       company.name AS name_of_company,
       funding_round.raised_amount AS amount
FROM company
JOIN funding_round ON company.id=funding_round.company_id
JOIN investment ON funding_round.id=investment.funding_round_id
JOIN fund ON investment.fund_id=fund.id
WHERE company.milestones>6
  AND EXTRACT(YEAR FROM CAST(funding_round.funded_at AS timestamp)) BETWEEN 2012 AND 2013;

**Задача 20**

Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT company_1.name AS acquiring_company,
       acquisition.price_amount,
       company_2.name AS acquired_company,
       company_2.funding_total,
       ROUND(acquisition.price_amount/company_2.funding_total) AS invest_percent
FROM acquisition
JOIN company AS company_1 ON acquisition.acquiring_company_id=company_1.id 
JOIN company AS company_2 ON acquisition.acquired_company_id=company_2.id
WHERE acquisition.price_amount!=0
  AND company_2.funding_total!=0
ORDER BY acquisition.price_amount DESC,
         acquired_company
LIMIT 10;

**Задача 21**

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT company.name AS company_name,
       EXTRACT(MONTH FROM CAST(funding_round.funded_at AS timestamp)) AS fund_month
FROM company
JOIN funding_round ON company.id=funding_round.company_id
WHERE funding_round.raised_amount!=0
  AND EXTRACT(YEAR FROM CAST(funding_round.funded_at AS timestamp)) BETWEEN 2010 AND 2013
  AND company.category_code='social';

**Задача 22**

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
i AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS timestamp)) AS month_acq,
             COUNT(acquired_company_id) AS count_acquired_company,
             SUM(price_amount) AS sum_price
      FROM acquisition
      WHERE EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN 2010 AND 2013
      GROUP BY month_acq),
y AS (SELECT EXTRACT(MONTH FROM CAST(funding_round.funded_at AS timestamp)) AS month_fund,
             COUNT(DISTINCT fund.name) AS count_fund_name
      FROM fund
      JOIN investment ON fund.id=investment.fund_id
      JOIN funding_round ON investment.funding_round_id=funding_round.id
      WHERE EXTRACT(YEAR FROM CAST(funded_at AS timestamp)) BETWEEN 2010 AND 2013
        AND fund.country_code = 'USA'
      GROUP BY month_fund )
SELECT month_fund,
       count_fund_name,
       count_acquired_company,
       sum_price
FROM i
JOIN y ON i.month_acq = y.month_fund

**Задача 23**

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
i AS (SELECT country_code,
             AVG(funding_total) AS avg_fund_2011
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = 2011
      GROUP BY country_code),
y AS (SELECT country_code,
             AVG(funding_total) AS avg_fund_2012
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = 2012 
      GROUP BY country_code),
e AS (SELECT country_code,
             AVG(funding_total) AS avg_fund_2013
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = 2013
      GROUP BY country_code)
SELECT i.country_code,
       avg_fund_2011,
       avg_fund_2012,
       avg_fund_2013
FROM i
JOIN y ON i.country_code=y.country_code
JOIN e ON e.country_code=y.country_code
ORDER BY avg_fund_2011 DESC;